In [9]:
%reset

In [10]:
import sys
sys.path.append('/home/users/ids29/DGRB')

In [11]:
import aegis
import numpy as np
import torch
import healpy as hp
import pickle as pk
from astropy import units
from astropy import constants as c
import matplotlib.pyplot as plt
from os import listdir
import os
from sbi.inference import SNLE, SNPE#, prepare_for_sbi, simulate_for_sbi
from sbi import utils as utils
from sbi import analysis as analysis
# from sbi.inference.base import infer
from getdist import plots, MCSamples
from joblib import Parallel, delayed
from scipy.integrate import quad, simpson
import pickle
from scipy.stats import norm
import sources.DMsignal as DMsignal

%matplotlib inline

In [12]:
grains=1000
num_simulations = 1000
num_workers = 48

In [13]:
parameter_range_aegis = [[], []]
abundance_luminosity_and_spectrum_list = []
source_class_list = []
parameter_names = []
energy_range = [1000, 100000] #MeV 
energy_range_gen = [energy_range[0]*0.5, energy_range[1]*18]
max_radius = 8.5 + 20*2 #kpc
exposure = 2000*10*0.2 #cm^2 yr
flux_cut = 1e-9 #photons/cm^2/s
angular_cut = np.pi #10*u.deg.to('rad') #degrees
angular_cut_gen = np.pi #angular_cut*1.5
lat_cut = 0 #2*u.deg.to('rad') #degrees
lat_cut_gen = lat_cut*0.5

In [14]:
my_cosmology = 'Planck18'
z_range = [0, 14]
luminosity_range = 10.0**np.array([37, 50]) # Minimum value set by considering Andromeda distance using Fermi as benchmark and receiving 0.1 photon at detector side
my_AEGIS = aegis.aegis(abundance_luminosity_and_spectrum_list, source_class_list, parameter_range_aegis, energy_range, luminosity_range, max_radius, exposure, angular_cut, lat_cut, flux_cut, energy_range_gen=energy_range_gen, cosmology = my_cosmology, z_range = z_range, verbose = False)
my_AEGIS.angular_cut_gen, my_AEGIS.lat_cut_gen = angular_cut_gen, lat_cut_gen

In [15]:
def DM_abundance(r, L, params): # Outputs the abundance function in the form dN/dV/dL. Note the dV.
    """
    Abundance function for the DM signal.
    """
    # r is the distance from the source # kpc
    # L is the luminosity of the source # photons/second
    # return the abundance function in the form dN/dV/dL
    # params conatins 'mass of DM particle in GeV'


    A_DM = params[0]
    mass_DM_GeV = params[2] # UPDAT TO THE CURRENT PARAMS INDEX

    
    beta = 1.9
    A = 1.2e4 # MSolar^{-1} kpc^{-3}
    r_s = 21 #kpc
    r_tilde = r / r_s # unitless
    cross_section = 3e-26 #cm^3 s^{−1}
    
    def compute_integral_E(E_min: float, E_max: float, n_points: int = 10_001) -> float:
        """
        Compute ∫ dN/dE dE between E_min and E_max [MeV]
        by sampling on a uniform grid and applying Simpson's rule.
        n_points must be odd for Simpson's rule.
        """
        # make sure we have an odd number of points
        if n_points % 2 == 0:
            n_points += 1

        E = np.linspace(E_min, E_max, n_points)
        dNdE = dm.get_dNdE(
            desired_E_MeV=E,
            channel=channel,
            mass_MeV= mass_DM_GeV * 1e3
        )
        return simpson(dNdE, E)

    Emin, Emax = energy_range[0], energy_range[1]
    integral_E = compute_integral_E(Emin, Emax)



    Phi_pp = A_DM * cross_section / (8 * np.pi * mass_DM_GeV**2) * integral_E  # cm^3 s^{-1} GeV^{-2}

    
    def integrand_M_tilde(M_tilde):
        sigma = 0.74 + 0.011 * np.log(r/50) - 0.003 * M_tilde / 0.87
        Delta = np.log(L) - 77.4 + 0.23*np.log(r/50) - np.log((8*np.pi*Phi_pp)/(1e-28)) # unitless
        mu = sigma**2 * (1-beta) / 0.87  +  Delta
        return np.exp((mu**2 -Delta**2)/(2*sigma**2)) * np.exp(-(M_tilde - mu)**2/(2*sigma**2)) / (np.sqrt(2*np.pi)*sigma)
    


    def compute_integral_M_tilde_simpson(M_min: float, M_max: float, n_points: int = 10001) -> float:
        """
        Integrate integrand_M_tilde from M_min to M_max using Simpson's rule.
        n_points must be odd; if it's even, it's incremented by 1.
        """
        if n_points % 2 == 0:
            n_points += 1
        M = np.linspace(M_min, M_max, n_points)
        y = integrand_M_tilde(M)
        return simpson(y, M)


    # Example usage:
    M_tilde_min = 0.87 * np.log(1/1e5) # M_min = 1 MSolar
    integral_M_tilde = compute_integral_M_tilde_simpson(M_tilde_min, 14.0) # The integrand 'integrand_M_tilde' is sufficiently suppressed for all M_tilde > 14.0



    
    return 10**(5*(1-beta)) * A / (0.87 * L * r_tilde * (1+r_tilde)**2)   * integral_M_tilde


def DM_abundance_dR(r, L, params): # Returns the abundance function in the form dN/dR/dL. Note the dR. # this function needs to be passed to AEGIS
    return 4*np.pi*r**2 * DM_abundance(r, L, params) # dV = 4*np.pi*r^2 dr, so we need to multiply by 4*np.pi*r^2


def DM_spectrum(energy, params): #'energy' is an array of energies in MeV 
    return dm.get_dNdE(
            desired_E_MeV=energy,
            channel=channel,
            mass_MeV= mass_DM_GeV * 1e3
        )

In [16]:
# als_Poisson = [spec_poisson] # belongs to 'isotropic_diffuse' class
als_DM = [DM_abundance_dR, DM_spectrum]
my_AEGIS.abun_lum_spec = [als_DM] # Change when you want to include Poisson model.
my_AEGIS.source_class_list = ['isotropic_faint_single_spectrum']

In [17]:
# a simple simulator with the total number of photons as the summary statistic
def simulator(params):

    input_params = params.numpy()

    source_info = my_AEGIS.create_sources(input_params, grains=grains, epsilon=1e-2)
    photon_info = my_AEGIS.generate_photons_from_sources(input_params, source_info, grains=grains) 
    obs_info = {'psf_fits_path': '/home/ids29/DGRB/FERMI_files/psf_P8R3_ULTRACLEANVETO_V2_PSF.fits', 'edisp_fits_path': '/home/ids29/DGRB/FERMI_files/edisp_P8R3_ULTRACLEANVETO_V2_PSF.fits', 'event_type': 'PSF3', 'exposure_map': None}
    obs_photon_info = my_AEGIS.mock_observe(photon_info, obs_info)
    
    return obs_photon_info

In [18]:
# 1) Point to your data directory and choose a channel
data_dir = "/home/users/ids29/DGRB/data/dm_spectra/"         # must contain AtProduction_gammas.dat
channel  = "Tau"                           # e.g. 'b' (bottom quarks) or 'Tau'


# 2) Build the DMsignal object
dm = DMsignal.DMsignal(directory=data_dir, channel=channel)

# ────────────────────────────────────────────────────────────────────────
# 3)  Ask the object for its default model parameters
#     (mass, amplitude, background).  These numbers come straight from
#     the get_default_model() method you showed.
# ────────────────────────────────────────────────────────────────────────
model_defaults = dm.get_default_model()        # {'A_DM': …, 'A_BG': …, 'mass_DM_MeV': …}
mass_DM_MeV_test    = model_defaults["mass_DM_MeV"]  # MeV
mass_DM_GeV_test    = mass_DM_MeV_test / 1e3                  # GeV
A_DM_test           = model_defaults["A_DM"] # unitless
A_BG_test           = model_defaults["A_BG"]

print(f"A_DM_test = {A_DM_test}; mass_DM_GeV_test = {mass_DM_GeV_test}; A_BG_test = {A_BG_test}")






A_DM_test = 200.0; mass_DM_GeV_test = 200.0; A_BG_test = 1.0


In [19]:
test_params = torch.tensor([A_DM_test, 0, mass_DM_GeV_test])
test_photon_info = simulator(test_params)

/tmp/ipykernel_583328/2559847122.py:48: RuntimeWarning: divide by zero encountered in log
  sigma = 0.74 + 0.011 * np.log(r/50) - 0.003 * M_tilde / 0.87


ValueError: operands could not be broadcast together with shapes (999,999) (10001,) 